In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum as spark_sum, avg as spark_avg
import pandas as pd
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Initialize Spark session
spark = SparkSession.builder \
    .appName("csv Preview Email with App Password") \
    .getOrCreate()

# Load csv table

gold_path_delta = '/Volumes/workspace/hls_demo_2/conform_data/gold_layer_delta'

# Load Gold layer data
gold_df = spark.read.format('delta').load(gold_path_delta)

agg_summary_df = gold_df.agg(
    spark_sum('total_sales_amount').alias('total_sales_amount'),
    spark_sum('total_sales_qty').alias('total_sales_qty'),
    spark_avg('avg_success_rate').alias('avg_success_rate'),
    spark_sum('total_stock_qty').alias('total_stock_qty')
)

agg_summary_pd = agg_summary_df.toPandas()
agg_summary_html = agg_summary_pd.to_html(index=False, border=1)


# Email details

my_email = "janakbhande111@gmail.com"
subject = "email notification check"
app_password = "crzvagfqtkufcodn"

# SMTP config (choose one)
use_gmail = True

smtp_server = "smtp.gmail.com" if use_gmail else "smtp.office365.com"
smtp_port = 587

# Construct email
msg = MIMEMultipart("alternative")
msg["Subject"] = subject
msg["From"] = my_email
msg["To"] = "janak.bhande2001@gmail.com" 

# HTML body
html_body = f"""
<html>
  <body>
    <h2>some aggregations in pipeline by Janak
    </h2>
    {agg_summary_html}
    <br><br>
  </body>
</html>
"""

msg.attach(MIMEText(html_body, "html"))

# Send email
try:
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(my_email, app_password)
        server.sendmail(my_email, "janak.bhande2001@gmail.com", msg.as_string())
    print("✅ Email sent successfully.")
except Exception as e:
    print(f"❌ Failed to send email: {e}")
